In [49]:
%pylab inline
from __future__ import division
import scipy as sp
from scipy import stats
import csv
from decimal import *
import mpld3
mpld3.enable_notebook()

Populating the interactive namespace from numpy and matplotlib


In [2]:
# from operator import mul    # or mul=lambda x,y:x*y
# from fractions import Fraction

import operator as op
def nCr(n, r):
    r = min(r, n-r)
    if r == 0: return 1
    numer = reduce(op.mul, xrange(n, n-r, -1))
    denom = reduce(op.mul, xrange(1, r+1))
    return numer//denom

# def nCr(n,r): 
#   return int( reduce(mul, (Fraction(n-i, i+1) for i in range(r)), 1) )

# def nCr(n,r):
#     f = math.factorial
#     return f(n) / f(r) / f(n-r)

# Use python round
# def round(x, n):
#     return int(np.rint(a * 10**n)) / 10.**n

In [3]:
# 4,3 Hamming Code - Reed Solomon Wrap
def hs_rs_table(rate, blocklength):
    k = (1-rate)*blocklength
    msg = rate*blocklength
    res = {}
    op_SNR = 1
    while op_SNR < 10**(5): # 10 = 100 dB
        pbitdrop = 0.5*math.erfc(sqrt(op_SNR/2))
        hcerr = 1 - ((1-pbitdrop)**7 + 7*pbitdrop*(1-pbitdrop)**6)
        hcf = 1 - (1-hcerr)**2
        reeddrop = sum([nCr(blocklength, d)* hcf**d *(1-hcf)**(blocklength-d) for d in range(int(k/2)+1, blocklength)])
        res[op_SNR] = reeddrop
        op_SNR *= 10**(0.01)
    return res
    
def print_table_csv(table, filename):
    ks = sorted(table.keys())
    vs = [table[k] for k in ks]
    with open(filename+'.csv', 'wb') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        writer.writerow(['SNR (dB)', 'P(fail)'])
        for rowid in xrange(len(ks)):
            writer.writerow([str(log10(ks[rowid])),str(vs[rowid])])
            

In [5]:
num_nodes = arange(1, 36, 1)
rates = num_nodes*160/10000
print(num_nodes)
print(rates)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25
 26 27 28 29 30 31 32 33 34 35]
[ 0.016  0.032  0.048  0.064  0.08   0.096  0.112  0.128  0.144  0.16
  0.176  0.192  0.208  0.224  0.24   0.256  0.272  0.288  0.304  0.32
  0.336  0.352  0.368  0.384  0.4    0.416  0.432  0.448  0.464  0.48
  0.496  0.512  0.528  0.544  0.56 ]


In [40]:
# Table Generation
test = hs_rs_table(35*160/10000, 256)
print_table_csv(test, 'lookup_tables/n35')

In [41]:
def load_table(filename):
    codetable = {}
    with open(filename, 'rb') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        for row in reader:
            if row[0] == 'SNR (dB)':
                continue
            codetable[Decimal(row[0])] = float(row[1])
    return codetable

def load_ordered_table(filename):
    codetable = []
    with open(filename, 'rb') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        for row in reader:
            if row[0] == 'SNR (dB)':
                continue
            codetable.append((float(row[0]), float(row[1])))
    return codetable

In [42]:
def lookup(codetable, kmin, kmax, snr):
    if snr < kmin: return 1.0
    elif snr > kmax: return 0.0
    return codetable[snr]

def snr_lookup(codetable, kmin, kmax, opSNR, dSNR):
    op_SNR = Decimal(str(opSNR))
    if op_SNR < kmin: return 1.0
    elif op_SNR > kmax: return 0.0
    if op_SNR in codetable:
        return codetable[op_SNR]
    lsnr = op_SNR.quantize(Decimal(str(dSNR)), rounding=ROUND_DOWN)
    rsnr = op_SNR.quantize(Decimal(str(dSNR)), rounding=ROUND_UP)
    lp, rp = lookup(codetable, kmin, kmax, lsnr), lookup(codetable, kmin, kmax, rsnr)
    return (rp-lp)*(opSNR-float(lsnr))/dSNR + lp

In [43]:
def p_single(codetable, kmin, kmax, op_SNR, endpoint, dfade):
    fadexp = sp.stats.expon()
    fade = arange(dfade, endpoint, dfade)
    return dfade*sum([snr_lookup(codetable, kmin, kmax, op_SNR+log10(f), 0.01) * fadexp.pdf(f) for f in fade])

In [44]:
def p_combo(codetable, kmin, kmax, a, op_SNR, endpoint, dfade):
    if a == 0:
        return 1.0
    fadexp = sp.stats.erlang(a)
    # 0.01 = dSNR for 4,3 Hamming Code table
    fade = arange(0, endpoint, dfade)
    return sum([snr_lookup(codetable, kmin, kmax, op_SNR+log10(f), 0.01) * fadexp.pdf(f) * dfade for f in fade])

In [45]:
# N = num_nodes
def p_protocol(codetable, kmin, kmax, N, op_SNR, endpoint, dfade):
    psingle = p_single(codetable, kmin, kmax, op_SNR, endpoint, dfade)
#     print('psingle', psingle, 'pcombo, ')
    return sum([nCr(N, a) * (1-psingle)**a * psingle**(N-a) * (1-(1-p_combo(codetable, kmin, kmax, a, op_SNR, endpoint, dfade))**(N-a)) for a in range(N)])

In [54]:
# Hockey Stick Plot

nominal_needed = []
for num_nodes in range(1, 35):
    filename = 'lookup_tables/n' + str(num_nodes) + '.csv'
    codetable = load_table(filename)
    SNR = 0.005
    pprotocol = 1.0
    while pprotocol > 10**(-4):
#         pprotocol = p_protocol(codetable, 0.0, 5.0, num_nodes, SNR, 1, 0.1)
        pprotocol = p_protocol(codetable, 0.0, 5.0, num_nodes, SNR, 10, 0.005)
    nominal_needed.append(SNR)


KeyboardInterrupt: 

In [53]:
print(nominal_needed)

[]


In [ ]:
plot(range(1, 35), nominal_needed, lw=2.0)
xlabel('Number of Nodes', fontsize=20)
ylabel('Nominal SNR Needed (dB)', fontsize=20)